# PyHawaii Lightning Talk: Data Validation with Pandera
- **Author:** Kaʻimi Kahihikolo, Data Scientist - Booz Allen Hamilton
- **Date:** 2024/04/11
- **Description:** Validating tabular data with the Python library: Pandera!

In [1]:
import pandas as pd
import pandera as pa

## The Problem Statement

As data scientists or engineers, we are often tasked with reading/processing external tabular data, such as csv, excel, etc. Depending on how this data is generated, or how delicate our downstream pipelines are, unforeseen changes in the source data may have dramatic downstream consequences in production environments.

Suppose we had the following scenario - this data could be uploaded to our web app, pulled via some api, etc.

In [2]:
good_df = pd.DataFrame({
    "Student_ID": ["ID_01", "ID_02", "ID_03"],
    "Score": [1, 3, 5]
})

print("Original table")
display(good_df)

# Split on _ and take 1th element
good_df["Student_ID"] = good_df["Student_ID"].apply(lambda x: int(x.split("_")[1]))
# Subtract 1 and cast to float
good_df["Score"] = (good_df["Score"] - 1).astype(float)

print("After 'cleaning' the data")
display(good_df)

Original table


,Student_ID,Score
0,ID_01,1
1,ID_02,3
2,ID_03,5


After 'cleaning' the data


,Student_ID,Score
0,1,0.0
1,2,2.0
2,3,4.0


We just built our data transformation pipeline with assumptions, like Student_ID is always `ID_{2 digit number}`. However, what if our input data changed - for example, a human error or a change in an automated process.


Here we define a bad dataframe which someone typod the Student ID and forgot to include the score column.

In [3]:
bad_df = pd.DataFrame({
    "Student_ID": ["ID_01", "ID_02", "ID_FISH_DOG"],
})

print("Original table")
display(bad_df)

print("Trying apply the same transformation code")
# Split on _ and take 1th element
bad_df["Student_ID"] = bad_df["Student_ID"].apply(lambda x: int(x.split("_")[1]))
# Subtract 1 and cast to float
bad_df["Score"] = (bad_df["Score"] - 1).astype(float)

Original table


,Student_ID
0,ID_01
1,ID_02
2,ID_FISH_DOG


Trying apply the same transformation code


ValueError: invalid literal for int() with base 10: 'FISH'

If this happens in a production environment, you may face dire consequences! For example, completely breaking your web app/product, ruining the customer experience, or even worse corrupting your production database.

Two questions:
1. Can we check our data before even performing transformations?
2. If the data doesn't conform to expectations, can we fail elegantly and log the reason?

## Pandera to the Rescue!

[Pandera](https://pandera.readthedocs.io/en/stable/index.html) is an open-source project which provides a convenient SDK to perform data validation on dataframe-like objects.

Easy to install python library:
```
pip install pandera
```

Optional extras:
```
pip install pandera[hypotheses]  # hypothesis checks
pip install pandera[io]          # yaml/script schema io utilities
pip install pandera[strategies]  # data synthesis strategies
pip install pandera[mypy]        # enable static type-linting of pandas
pip install pandera[fastapi]     # fastapi integration
pip install pandera[dask]        # validate dask dataframes
pip install pandera[pyspark]     # validate pyspark dataframes
pip install pandera[modin]       # validate modin dataframes
pip install pandera[modin-ray]   # validate modin dataframes with ray
pip install pandera[modin-dask]  # validate modin dataframes with dask
pip install pandera[geopandas]   # validate geopandas geodataframes
```

In [4]:
# Load in the Pandera library
import pandera as pa

# Example bad data
bad_df = pd.DataFrame({
    "Student_ID": ["ID_01", "ID_02", "ID_FISH_DOG"],
})
print("Bad data")
display(bad_df)

Bad data


,Student_ID
0,ID_01
1,ID_02
2,ID_FISH_DOG


### Simple Data Type Check

Let's just check if the data types of the columns are right and if the columns are present.

In [5]:
simple_schema = pa.DataFrameSchema({
    "Student_ID": pa.Column(str, required=True),
    "Score": pa.Column(int, required=True),
})

In [6]:
print("Validate bad data")
display(simple_schema.validate(bad_df))

Validate bad data


SchemaError: column 'Score' not in dataframe. Columns in dataframe: ['Student_ID']

### Making it more robust with checks

Let's apply slighlty more rigorous checks to make sure the data follows our expectations!

In [7]:
schema_robust = pa.DataFrameSchema({
    "Student_ID": pa.Column(
        str,
        required=True,
        checks=[
            # Apply regex match
            pa.Check.str_matches("ID_[0-9]{2}")
        ]
    ),
    "Score": pa.Column(
        int,
        required=True,
        checks=[
            # Less than or equal to 5
            pa.Check.le(5),
            # Greater than 0
            pa.Check.gt(0)
        ]
    ),
})

When we validate our data, let's also have Python create a nice JSON representation of the error for logging purposes!

In [8]:
import json

try:
    schema_robust.validate(bad_df, lazy=True)
except pa.errors.SchemaErrors as e:
    print(json.dumps(e.message, indent=4))

{
    "SCHEMA": {
        "COLUMN_NOT_IN_DATAFRAME": [
            {
                "schema": null,
                "column": null,
                "check": "column_in_dataframe",
                "error": "column 'Score' not in dataframe. Columns in dataframe: ['Student_ID']"
            }
        ]
    },
    "DATA": {
        "DATAFRAME_CHECK": [
            {
                "schema": null,
                "column": "Student_ID",
                "check": "str_matches('ID_[0-9]{2}')",
                "error": "Column 'Student_ID' failed element-wise validator number 0: str_matches('ID_[0-9]{2}') failure cases: ID_FISH_DOG"
            }
        ]
    }
}


You can find a full list of supported checks on their documentation: https://pandera.readthedocs.io/en/stable/checks.html

What this means:
- Data engineers can build these checks at the beginning of your data pipelines and handle the errors accordingly so your production environment doesn't break.
- Upon error, record WHY it failed so you can fix the issue later!

## Bonus: Synthetic Data Generation

Once you have your schema defined, it is really easy to tell Pandera to generate synthetic data for you! This is very useful for unit testing your code or developing code when you don't have access to the actual data.

In [9]:
schema_robust = pa.DataFrameSchema({
    "Student_ID": pa.Column(
        str,
        required=True,
        checks=[
            # Apply regex match
            pa.Check.str_matches("ID_[0-9]{2}")
        ]
    ),
    "Score": pa.Column(
        int,
        required=True,
        checks=[
            # Less than or equal to 5
            pa.Check.le(5),
            # Greater than 0
            pa.Check.gt(0)
        ]
    ),
})

print("New synthesized data!")
display(schema_robust.example(size=5))

New synthesized data!


,Student_ID,Score
0,ID_77,2
1,ID_35,2
2,ID_87,5
3,ID_52,2
4,ID_95,3


In summary, Pandera is great and you should check it out!